In [160]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [161]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from urllib.request import Request, urlopen

import os
import json
import time
import random
import re

In [162]:
import undetected_chromedriver as uc

In [163]:
PATH = "C:\\chrome_driver\\chromedriver.exe"

page_url = 'https://codeforces.com/problemset/page/8'


In [164]:
req = Request(
    url=page_url, 
    headers={'User-Agent': 'Mozilla/5.0'}
)
problem_page_html = urlopen(req).read()

soup = BeautifulSoup(problem_page_html, 'lxml')

with open('out.html', 'w') as f:
    f.write(soup.prettify())

In [165]:
name_prblms = []

In [166]:
ques = soup.find_all('td', class_ = 'id')
extracted_links = [link.a.text.replace(' ','').strip() for link in ques if link.a]

for link in extracted_links:
    name_prblms.append(link)
    print(link)

1949F
1949E
1949D
1949C
1949B
1949A
1948G
1948F
1948E
1948D
1948C
1948B
1948A
1946F
1946E
1946D
1946C
1946B
1946A
1945H
1945G
1945F
1945E
1945D
1945C
1945B
1945A
1944B
1944A
1943F
1943E2
1943E1
1943D2
1943D1
1943C
1943B
1943A
1942H
1942G
1942F
1942E
1942D
1942C2
1942C1
1942B
1942A
1941G
1941F
1941E
1941D
1941C
1941B
1941A
1940D
1940C
1940B
1940A
1939D
1939C
1939B
1939A
1938M
1938L
1938K
1938J
1938I
1938H
1938G
1938F
1938E
1938D
1938C
1938B
1938A
1937B
1937A
1936F
1936E
1936D
1936C
1936B
1936A
1935F
1935E
1935D
1935C
1935B
1935A
1934E
1934D2
1934D1
1934C
1934B
1934A
1933G
1933F
1933E
1933D
1933C
1933B


In [167]:
# name_prblms = name_prblms[25:]

In [168]:
import os
import json

# Define directories for storage
TEXT_DIR = "problems_text"
JSON_DIR = "problems_metadata"
os.makedirs(TEXT_DIR, exist_ok=True)
os.makedirs(JSON_DIR, exist_ok=True)

def store_problem(problem_data):
    """
    Store the problem statement and metadata.

    :param problem_data: Dictionary containing problem details:
        {
            "title": "Problem Title",
            "time_limit": "2 seconds",
            "memory_limit": "256 MB",
            "statement": "Problem statement here",
            "tags": ["tag1", "tag2"],
            "solution": "solution to the question"
        }
    """
    try:
        # Extract details from problem_data
        title = problem_data.get("title")
        statement = problem_data.get("statement")
        metadata = {
            "time_limit": problem_data.get("time_limit"),
            "memory_limit": problem_data.get("memory_limit"),
            "tags": problem_data.get("tags", []),
            "solution": problem_data.get("solution")
        }

        # File-safe title for storage
        file_safe_title = "_".join(title.split()).replace("/", "-")

        # Save the problem statement as a text file
        text_file_path = os.path.join(TEXT_DIR, f"{file_safe_title}.txt")
        with open(text_file_path, "w", encoding="utf-8") as text_file:
            text_file.write(statement)

        # Save the metadata as a JSON file
        json_file_path = os.path.join(JSON_DIR, f"{file_safe_title}.json")
        with open(json_file_path, "w", encoding="utf-8") as json_file:
            json.dump(metadata, json_file, indent=4)

        print(f"Stored problem: {title}")

    except Exception as e:
        print(f"Error storing problem {problem_data.get('title', 'unknown')}: {e}")

# Example usage
# problem_example = {
#     "title": "Sample Problem",
#     "time_limit": "2 seconds",
#     "memory_limit": "256 MB",
#     "statement": "This is the problem statement.",
#     "tags": ["implementation", "math"]
# }

# store_problem(problem_example)

In [169]:
from bs4 import BeautifulSoup

def get_solution(driver, problem_code):
    page_html = driver.page_source
    
    soup = BeautifulSoup(page_html, 'lxml')
    
    solutions = []
    for p_tag in soup.find_all('p'):
        if problem_code in p_tag.get_text():
            # Check the next sibling for the div with class "spoiler"
            spoiler_div = p_tag.find_next_sibling('div', class_='spoiler')
            if spoiler_div:
                # Find the <b> tag with class "spoiler-title"
                spoiler_title = spoiler_div.find('b', class_='spoiler-title')
                if spoiler_title and spoiler_title.get_text().strip().lower() in {'editorial', 'tutorial', 'solution'}:
                    # Extract all <p> tags within the div with class "spoiler-content"
                    spoiler_content = spoiler_div.find('div', class_='spoiler-content')
                    if spoiler_content:
                        paragraphs = [p.get_text().strip() for p in spoiler_content.find_all('p')]
                        solutions.extend(paragraphs)
                    
    if solutions:
        print(solutions)
        return solutions  # Return only the content of the first match
    else:
        return None


In [170]:
from bs4 import BeautifulSoup
import re

def extract_problem_statement_until(tag, stop_class):
    """
    Extracts all text paragraphs after the memory limit section until a specific class is encountered.
    Stops when encountering a div with the specified class.
    Ensures that missing content is replaced with a single whitespace.
    """
    problem_statements = []
    for sibling in tag.find_all_next():
        # Stop if the specified class is encountered
        if sibling.name == 'div' and sibling.get('class') and stop_class in sibling.get('class'):
            break
        if sibling.name == 'p':  # Collect paragraphs
            text = sibling.get_text(strip=True, separator=' ')
            # Replace empty strings or excessive whitespace with a single space
            problem_statements.append(text)
    return "\n".join(problem_statements)

# Load HTML file
# html_file_path = r"ques_html\3.html"
def save_content_of_question(content):
    soup = BeautifulSoup(content, 'lxml')

    # Extract problem title
    problem_title_tag = soup.find('div', class_='title')
    if problem_title_tag is None:
        return
    problem_title = ' '.join(problem_title_tag.text.split()[1:])

    # Extract time limit and memory limit
    time_limit = soup.find('div', class_='time-limit').contents[-1].strip()
    memory_limit = soup.find('div', class_='memory-limit').contents[-1].strip()

    # Extract problem statement starting from memory limit
    memory_limit_tag = soup.find('div', class_='memory-limit')
    problem_text = (
        extract_problem_statement_until(memory_limit_tag, 'input-specification')
        if memory_limit_tag
        else "N/A"
    )
    
    question_tag_tag = soup.find_all('span', class_='tag-box')
    question_tags = []
    
    if(question_tag_tag):
        for tag in question_tag_tag:
            question_tags.append(tag.text.strip())

    # Display extracted information
    print(f"Title: {problem_title}")
    print(f"Time Limit: {time_limit}")
    print(f"Memory Limit: {memory_limit}")
    print(f"Problem Statement:\n{problem_text}")
    print("Problem tags: ", end=' ')
    for tag in question_tags:
        print(tag, end=", ")
        
    print('')

    #SAVE IN TXT AND JSON    
    problem_data = {
        "title": problem_title,
        "time_limit": time_limit,
        "memory_limit": memory_limit,
        "statement": problem_text,
        "tags": question_tags
    }
    
    # store_problem(problem_data)
    
    return problem_data

    # Save problem statement to a file
    # output_file_path = r"ques_html\problem_statement.txt"
    # with open(output_file_path, 'w', encoding='utf-8') as file:
    #     file.write(problem_text)

In [171]:
def save_problem_html(driver, index):
    page_source = driver.page_source
    
    return save_content_of_question(page_source)

In [172]:
def setup_driver():
    driver = uc.Chrome(use_subprocess=True)
    return driver

def human_like_delay():
    time.sleep(random.uniform(2, 4))

def is_window_open(driver):
    try:
        driver.current_window_handle
        return True
    except:
        return False

def scrape_with_captcha_handling(name_prblms):
    driver = setup_driver()
    try:
        driver.get(page_url)
        human_like_delay()

        for index, text in enumerate(name_prblms):
            if not is_window_open(driver):
                print("Browser window closed unexpectedly.")
                break
            try:
                # Locate the problem link
                
                print('locating problem link')
                
                element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.LINK_TEXT, text))
                )
                driver.execute_script("arguments[0].scrollIntoView(true);", element)
                human_like_delay()

                link = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, text))
                )
                link_ques = link.get_attribute('href')
                link.click()
                human_like_delay()
                
                print('entered problem page')

                problem_code = text

                print(problem_code)


                # Save the problem HTML
                data = save_problem_html(driver, index)
                
                print('got data except solution')
                
                human_like_delay()
                
                print('finding tutorial link')

                # Find and click the tutorial/editorial link
                tutorial_link = driver.find_element(
                    By.XPATH,
                    "//a[not(contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'video')) and (contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'solution') or contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'tutorial') or contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'editorial'))]"
                )

                tutorial_link.click()
                human_like_delay()

                # Handle the new tab
                original_window = driver.current_window_handle
                new_window = [window for window in driver.window_handles if window != original_window][0]
                driver.switch_to.window(new_window)
                
                # SAVE SOLUTION FROM EDITORIAL PAGE
                
                edit_page = driver.page_source
                soup = BeautifulSoup(edit_page, 'lxml')
                with open ('ques_html/editorial.html' , 'w') as f:
                    f.write(soup.prettify())

                # Perform any operations on the new tab
                print("On the tutorial/editorial page.")
                
                human_like_delay()
                
                solution = get_solution(driver, problem_code)
                data["solution"] = solution
                
                store_problem(data)
                
                # Close the new tab and switch back
                driver.close()
                driver.switch_to.window(original_window)

                # Return to the problem page
                driver.get(link_ques)
                driver.back()
                human_like_delay()

            except Exception as e:
                print(f"Error with problem '{text}': {e}")
                driver.back()
                human_like_delay()
                continue
    finally:
        if is_window_open(driver):
            driver.quit()

# Usage
scrape_with_captcha_handling(name_prblms)

locating problem link
entered problem page
1945B
Title: Fireworks
Time Limit: 1 second
Memory Limit: 256 megabytes
Problem Statement:
One of the days of the hike coincided with a holiday, so in the evening at the camp, it was decided to arrange a festive fireworks display. For this purpose, the organizers of the hike bought two installations for launching fireworks and a huge number of shells for launching.
Both installations are turned on simultaneously. The first installation launches fireworks every a a minutes (i.e., after a , 2 ⋅ a , 3 ⋅ a , … a , 2 ⋅ a , 3 ⋅ a , … minutes after launch). The second installation launches fireworks every b b minutes (i.e., after b , 2 ⋅ b , 3 ⋅ b , … b , 2 ⋅ b , 3 ⋅ b , … minutes after launch).
Each firework is visible in the sky for m + 1 m + 1 minutes after launch, i.e., if a firework was launched after x x minutes after the installations were turned on, it will be visible every minute from x x to x + m x + m , inclusive. If one firework was launc